In [1]:
import pypsa

/Users/gbotemi/miniconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


In [2]:
# initialize network
n = pypsa.Network()

/Users/gbotemi/miniconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/Users/gbotemi/miniconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
/Users/gbotemi/miniconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas.

In [3]:
# add carriers
n.add("Carrier", "heat")
n.add("Carrier", "AC")
	

In [4]:

# add buses
n.add("Bus", "csp_internal_bus", carrier="heat", v_nom=20.0)
n.add("Bus", "electricity_bus", carrier="AC", v_nom=18.0)

In [5]:
n.add("Store", "salt_tank", bus="csp_internal_bus", standing_loss=0.5, marginal_cost=2000)

In [6]:
# add generator and store
n.add("Generator", "receiver", bus="csp_internal_bus", p_set=100, marginal_cost=2000)

In [7]:

# add links
n.add("Link", "heat_exchanger", bus0="csp_internal_bus", bus1="electricity_bus", efficiency=0.6)

In [8]:

#add another generator for the electricity bus
n.add("Generator", "steam_electricity", bus="electricity_bus", p_set=100, marginal_cost=7000)


In [9]:

# add load
n.add("Load", "external_load", bus="electricity_bus", p_set=100)

In [10]:
n

PyPSA Network
Components:
 - Bus: 2
 - Carrier: 2
 - Generator: 2
 - Link: 1
 - Load: 1
 - Store: 1
Snapshots: 1

In [12]:
n.optimize()

/Users/gbotemi/miniconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/optimization/constraints.py:516: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  (-get_as_dense(n, "Load", "p_set", sns) * n.loads.sign)
INFO:linopy.model: Solve problem using Glpk solver
INFO:linopy.io: Writing time: 0.02s
Status: warning
Termination condition: infeasible_or_unbounded
Solution: 0 primals, 0 duals
Objective: nan
Solver model: not available
Solver message: undefined



GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --lp /var/folders/c3/657p__892k72qxh6r8fdkqsh0000gn/T/linopy-problem-tag9gein.lp
 --output /var/folders/c3/657p__892k72qxh6r8fdkqsh0000gn/T/linopy-solve-kbuil1ov.sol
Reading problem data from '/var/folders/c3/657p__892k72qxh6r8fdkqsh0000gn/T/linopy-problem-tag9gein.lp'...
11 rows, 5 columns, 15 non-zeros
61 lines were read
GLPK Simplex Optimizer 5.0
11 rows, 5 columns, 15 non-zeros
Preprocessing...
PROBLEM HAS NO PRIMAL FEASIBLE SOLUTION
If you need actual output for non-optimal solution, use --nopresol
Time used:   0.0 secs
Memory used: 0.0 Mb (32525 bytes)
Writing basic solution to '/var/folders/c3/657p__892k72qxh6r8fdkqsh0000gn/T/linopy-solve-kbuil1ov.sol'...


('warning', 'infeasible_or_unbounded')